In [2]:
import pandas as pd
from pyspark.sql import SparkSession

In [3]:
from pyspark.sql.functions import udf
from bs4 import BeautifulSoup

In [4]:
from pyspark.sql.functions import regexp_replace

In [5]:
import html2text
from pyspark.sql.functions import col

In [6]:
file_path = "/home/dhwang/Project2/propublica_1000.csv"

spark = SparkSession.builder.master("local[*]").appName("Prep").getOrCreate()

df = spark.read.csv(file_path, header=True, inferSchema=True, multiLine=True, sep=',', escape='"',
                    ignoreLeadingWhiteSpace=True)

23/05/10 14:49:37 WARN Utils: Your hostname, DSGPU05 resolves to a loopback address: 127.0.1.1; using 10.10.11.64 instead (on interface eno1)
23/05/10 14:49:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/10 14:49:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=66>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/loca

Py4JError: An error occurred while calling o35.csv

In [7]:
df.createOrReplaceTempView("tempfile")

new_df = spark.sql("select id, message from tempfile")

In [75]:
new_df.show(5)

+-------------+--------------------+---------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-----------+---------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+------------------------------+
|           id|                html|political|not_political|               title|             message|           thumbnail|          created_at|          updated_at| lang|              images|impressions|political_probability|           targeting|suppressed|             targets|          advertiser|            entities|                page|          lower_page|          targetings|         paid_for_by|targetedness|listbuilding_fundraising_proba|
+-------------+--------------------+---------+-------------+--------------------+-------------------

In [9]:
new_df.select("message").take(5)

[Row(message='<p>1 in 10 Coloradans don’t know where they will get their next meal. We can and will do better if everyone joins together. Join the effort to end hunger in our state.</p>'),
 Row(message='<p>Sabah and her family were forced to flee their home when violence threatened their lives.<br> Will you help them?</p>'),
 Row(message="<p>So many of the ocean's most majestic animals are seriously endangered. If we don't act, we could lose many of them forever…</p>"),
 Row(message='<p><span class=""_5mfr _47e3""><img class=""img"" src=""https://pp-facebook-ads.s3.amazonaws.com/images/emoji.php/v9/f45/1/16/1f6a8.png""><span class=""_7oe"">🚨</span></span>Breaking: North Pole hits record high temperatures<span class=""_5mfr _47e3""><img class=""img"" src=""https://pp-facebook-ads.s3.amazonaws.com/images/emoji.php/v9/f45/1/16/1f6a8.png""><span class=""_7oe"">🚨</span></span></p><p> We cannot afford to stand by and let the Trump administration push reckless climate change-denying policies.

grab texts only

In [10]:
def html_to_text(html):
    soup = BeautifulSoup(html, 'html.parser')
    return soup.get_text()

In [11]:
html_to_text_udf = udf(html_to_text)

In [76]:
df2 = new_df.withColumn("message", html_to_text_udf(df["message"]))

In [77]:
df2.show(5)

+-------------+--------------------+---------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-----------+---------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+------------------------------+
|           id|                html|political|not_political|               title|             message|           thumbnail|          created_at|          updated_at| lang|              images|impressions|political_probability|           targeting|suppressed|             targets|          advertiser|            entities|                page|          lower_page|          targetings|         paid_for_by|targetedness|listbuilding_fundraising_proba|
+-------------+--------------------+---------+-------------+--------------------+-------------------

getting word length and word counts

In [78]:
from pyspark.sql.functions import udf, col, size, regexp_replace, split, expr
from pyspark.sql.types import IntegerType, FloatType

# Function to count the number of words in a string
def count_words(text):
    if text is not None:
        return len(text.split())
    else:
        return 0

# Define UDF for counting words
count_words_udf = udf(count_words, IntegerType())

# Add new column for word count
df3 = df2.withColumn("word_count", count_words_udf("message"))

# Remove non-word characters and split the message into words
df3 = df3.withColumn("words", split(regexp_replace(col("message"), "[^\w\s]", ""), " "))

# Calculate the average word length
df3 = df3.withColumn("avg_word_length", expr("aggregate(words, 0, (acc, word) -> acc + length(word), acc -> acc / size(words))"))

df3.show()


23/05/08 09:49:02 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------------+--------------------+---------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-----------+---------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+------------------------------+----------+--------------------+------------------+
|               id|                html|political|not_political|               title|             message|           thumbnail|          created_at|          updated_at| lang|              images|impressions|political_probability|           targeting|suppressed|             targets|          advertiser|            entities|                page|          lower_page|          targetings|         paid_for_by|targetedness|listbuilding_fundraising_proba|word_count|               words|   avg_wor

sentiment with df3

In [70]:
import sys
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import col, udf, explode, count, row_number, when, avg, round
from pyspark.sql.types import BooleanType, FloatType
from pyspark.sql.window import Window
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [71]:
def sentiment_score(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment = analyzer.polarity_scores(text)
    return sentiment["compound"]

sentiment_score_udf = udf(sentiment_score, returnType=FloatType())

In [72]:
df4 = df3.withColumn("sentiment_score", sentiment_score_udf("message"))

In [73]:
df4.show(5)

+-------------+--------------------+----------+--------------------+-----------------+---------------+
|           id|             message|word_count|               words|  avg_word_length|sentiment_score|
+-------------+--------------------+----------+--------------------+-----------------+---------------+
|6111675777657|1 in 10 Coloradan...|        32|[1, in, 10, Color...|          4.03125|         0.4767|
|6093230890253|Sabah and her fam...|        19|[Sabah, and, her,...|4.684210526315789|        -0.8126|
|6106277477628|So many of the oc...|        22|[So, many, of, th...|4.409090909090909|        -0.5267|
|6093719995084|🚨Breaking: North...|        33|[Breaking, North,...|5.242424242424242|         0.1531|
|6119568277711|You deserve a med...|       108|[You, deserve, a,...|4.486238532110092|        -0.6408|
+-------------+--------------------+----------+--------------------+-----------------+---------------+
only showing top 5 rows



In [83]:
from pyspark.sql.functions import col, concat_ws

df5 = df4.withColumn("words", concat_ws(",", col("words").cast("string")))

In [84]:
df5.show(5)

+-------------+--------------------+----------+--------------------+-----------------+---------------+
|           id|             message|word_count|               words|  avg_word_length|sentiment_score|
+-------------+--------------------+----------+--------------------+-----------------+---------------+
|6111675777657|1 in 10 Coloradan...|        32|[1, in, 10, Color...|          4.03125|         0.4767|
|6093230890253|Sabah and her fam...|        19|[Sabah, and, her,...|4.684210526315789|        -0.8126|
|6106277477628|So many of the oc...|        22|[So, many, of, th...|4.409090909090909|        -0.5267|
|6093719995084|🚨Breaking: North...|        33|[Breaking, North,...|5.242424242424242|         0.1531|
|6119568277711|You deserve a med...|       108|[You, deserve, a,...|4.486238532110092|        -0.6408|
+-------------+--------------------+----------+--------------------+-----------------+---------------+
only showing top 5 rows



In [85]:
df5.write.csv("test1000.csv", header=True)

targets tinkering

In [21]:
targets_only = pd.read_csv("targets_only.csv")
publica = pd.read_csv("fbpac-ads-en-US.csv")

In [22]:
publica

,id,html,political,not_political,title,message,thumbnail,created_at,updated_at,lang,...,suppressed,targets,advertiser,entities,page,lower_page,targetings,paid_for_by,targetedness,listbuilding_fundraising_proba
0,hyperfeed_story_id_5c9baa3ee0ec08073500042,"<div class=""_5pa- userContentWrapper""><div cla...",0,0,League of Conservation Voters,<p>BREAKING: Trump’s Department of the Interio...,https://pp-facebook-ads.s3.amazonaws.com/v/t1....,2019-03-27 16:52:25.625455+00,2019-03-27 16:52:25.625455+00,en-US,...,f,[],NaN,"[{""entity"": ""Endangered Species Act"", ""entity_...",https://www.facebook.com/LCVoters/,https://www.facebook.com/lcvoters/,NaN,League of Conservation Voters,NaN,0.647945
1,hyperfeed_story_id_5c9bb2a2413852086735771,"<div class=""_5pa- userContentWrapper""><div cla...",0,0,Indivisible Guide,<p>The Mueller investigation is over. Special ...,https://pp-facebook-ads.s3.amazonaws.com/v/t1....,2019-03-27 17:28:14.096849+00,2019-03-27 17:28:14.096849+00,en-US,...,f,[],NaN,"[{""entity"": ""Americans"", ""entity_type"": ""Group...",https://www.facebook.com/indivisibleguide/,https://www.facebook.com/indivisibleguide/,NaN,Indivisible Project,NaN,0.350635
2,hyperfeed_story_id_5c9bb4fa461731e29426627,"<div class=""_5pa- userContentWrapper""><div cla...",0,0,International Rescue Committee,<p>Zimbabwe is reeling from the impact of Cycl...,https://pp-facebook-ads.s3.amazonaws.com/v/t1....,2019-03-27 17:38:23.101377+00,2019-03-27 17:38:23.101377+00,en-US,...,f,[],NaN,"[{""entity"": ""Zimbabwe"", ""entity_type"": ""Region""}]",https://www.facebook.com/InternationalRescueCo...,https://www.facebook.com/internationalrescueco...,NaN,International Rescue Committee,NaN,0.999909
3,23843380741530360,"<div class=""_5pcr userContentWrapper""><div cla...",0,0,Covenant House International,<p>What more can you do in the final hours of ...,https://pp-facebook-ads.s3.amazonaws.com/v/t1....,2018-12-30 20:59:13.879124+00,2018-12-30 20:59:13.879124+00,en-US,...,f,"[{""target"": ""Activity on the Facebook Family""}...",Covenant House International,[],https://www.facebook.com/CovenantHouse/,https://www.facebook.com/covenanthouse/,"{""<div><div class=\""_4-i0 _26c5\""><div class=\...",Covenant House International,5.0,NaN
4,hyperfeed_story_id_5c9bb059454851c17741213,"<div class=""_5pa- userContentWrapper""><div cla...",0,1,Planned Parenthood,"<p>Say it loud, say it proud: Our rights, our ...",https://pp-facebook-ads.s3.amazonaws.com/v/t1....,2019-03-27 17:18:29.764002+00,2019-04-11 15:02:58.081112+00,en-US,...,f,[],NaN,"[{""entity"": ""Planned Parenthood"", ""entity_type...",https://www.facebook.com/PlannedParenthood/,https://www.facebook.com/plannedparenthood/,NaN,Planned Parenthood Federation of America,NaN,0.999977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162319,23843108782710078,"<div class=""_5pcr userContentWrapper""><div cla...",12,0,Keep Them Accountable,<p>Rep. Katko voted for tax breaks for his wea...,https://pp-facebook-ads.s3.amazonaws.com/v/t1....,2018-10-19 10:31:52.466563+00,2018-10-22 11:40:06.24382+00,en-US,...,f,"[{""target"": ""Age"", ""segment"": ""18 and older""},...",Keep Them Accountable,"[{""entity"": ""Katko"", ""entity_type"": ""Person""}]",https://www.facebook.com/KeepThemAccountable18/,https://www.facebook.com/keepthemaccountable18/,"{""<div><div class=\""_4-i0 _26c5\""><div class=\...","HOUSE MAJORITY PAC, (202) 849-6052, AND PRIORI...",7.0,0.116965
162320,23843034525850259,"<div class=""_5pcr userContentWrapper""><div cla...",0,0,National Republican Congressional Committee,<p>Illinois early voting is open NOW &amp; you...,https://pp-facebook-ads.s3.amazonaws.com/v/t1....,2018-10-24 20:41:42.111865+00,2018-10-24 20:41:42.111865+00,en-US,...,f,"[{""target"": ""List""}, {""target"": ""Age"", ""segmen...",National Republican Congressional Committee,"[{""entity"": ""Illinois"", ""entity_type"": ""Region...",https://www.facebook.com/NRCC/,https://www.facebook.com/nrcc/,"{""<div><d

In [17]:
targets_only

,row_id,target_Age,target_Agency,target_City,target_Gender,target_Interest,target_Language,target_Like,target_List,target_MaxAge,target_MinAge,target_Region,target_Retargeting,target_Segment,target_State,target_Website
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,women,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34 and older,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34,NaN,NaN,NaN,NaN
4,4,18 and older,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484027,484027,35 to 54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484028,484028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,NaN,NaN,NaN,NaN,NaN
484029,484029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.0,NaN,NaN,NaN,NaN,NaN,NaN
484030,484030,NaN,NaN,NaN,men,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
import json
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import  *
import re


def quoter(row):
    """
    Takes a JSON row and will replace the occurrences of double-double quotes ("") with a single double quote (")
    :param row: A cell from the Targets Column
    :return: a valid JSON format String
    """
    return [row[0], re.sub('""', '"', row[1])]


def col_name_extractor(row):
    row1 = json.loads(row[1])
    result = []
    for item in row1:
        if " " not in item["target"]:
            result.append("target" + "_" + item["target"])
    return [row[0], result]


def value_extractor(row):
    row1 = json.loads(row[1])
    result = []
    for item in row1:
        try:
            result.append(item["segment"])
        except:
            result.append(" ")
    return result


def flattner(rdd):
    """

    :param rdd: the RDD Of either the Columns or the Values extracted from the targets column
    :return: A flat list of those values
    """
    flattened_rdd = rdd.flatMap(lambda x: x[1])
    unique_set = set(flattened_rdd.collect())

    return list(unique_set)


def json_ready(df):
    """
    Takes the Targets Columns and returns in
    :param df: The Dataframe of the targets column
    :return: a cleaned up and JSON processing ready PySpark RDD of that column
    """
    rdd = df.select("targets").rdd
    rdd2 = rdd.zipWithIndex().map(lambda x: (x[1], x[0][0]))
    rdd3 = rdd2.map(quoter)

    return rdd3

In [20]:
file_path = "/home/dhwang/Project2/propublica_1000.csv"

spark = SparkSession.builder \
    .master('local[1]') \
    .getOrCreate()
df = spark.read.csv(file_path, header=True, inferSchema=True, multiLine=True, sep=',', escape='"',
                    ignoreLeadingWhiteSpace=True)
df.select("targets").show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|targets                                                                                                                                                                                                                                                                         |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[{""target"": ""Age"", ""segment"": ""30 and older""}, {""target"": ""MinAge"", ""segment"": ""30""}, {""target"": ""Region"", ""segment"": ""Colorado""}, {""target"": ""Lang

In [23]:
df.show(5)

+-------------+--------------------+---------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-----------+---------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+------------------------------+
|           id|                html|political|not_political|               title|             message|           thumbnail|          created_at|          updated_at| lang|              images|impressions|political_probability|           targeting|suppressed|             targets|          advertiser|            entities|                page|          lower_page|          targetings|         paid_for_by|targetedness|listbuilding_fundraising_proba|
+-------------+--------------------+---------+-------------+--------------------+-------------------

In [6]:
jsoned_rdd = json_ready(df)

cols = jsoned_rdd.map(col_name_extractor)  # 2D RDD of the columns per cell
flat_cols_unique_list = flattner(cols)     # list of unique column names
print(flat_cols_unique_list)

['target_State', 'target_Language', 'target_List', 'target_MaxAge', 'target_Website', 'target_Age', 'target_City', 'target_Region', 'target_Agency', 'target_Segment', 'target_Like', 'target_Interest', 'target_MinAge', 'target_Retargeting', 'target_Gender']


In [7]:
"""# Add the unique columns as empty columns to the Dataframe
for col_name in flat_cols_unique_list:
    df = df.withColumn(col_name, lit(" "))"""

vals = jsoned_rdd.map(value_extractor)     # 2D RDD of the values per cell
print(vals.take(5))
print("\n")
columns_list = cols.collect()
values_list = vals.collect()

[['30 and older', '30', 'Colorado', 'English (US)'], ['18 and older', '18', 'people who may be similar to their customers', 'the United States'], ['people who have visited their website or used one of their apps', '18 and older', '18', 'the United States'], ['18 and older', '18', 'people who may be similar to their customers', 'the United States'], ['18 and older', '18', 'the United States', ' ']]




In [32]:
rdd = df.rdd
modified_rdd = rdd
# TODO: Turn this bit into MAP function
# Create an empty dataframe
new_rows = []
for item in columns_list:
    column_index = item[0]
    col_group = item[1]
    for index2, col_name in enumerate(col_group):
        row_index = index2
        value = values_list[column_index][row_index]
        new_row = Row(col_name=col_name, value=value)
        new_rows.append(new_row)

# Create a new DataFrame using the new rows
df = spark.createDataFrame(new_rows)

Row(col_name='target_Age', value='30 and older')
Row(col_name='target_MinAge', value='30')
Row(col_name='target_Region', value='Colorado')
Row(col_name='target_Language', value='English (US)')
Row(col_name='target_Age', value='18 and older')
Row(col_name='target_MinAge', value='18')
Row(col_name='target_Retargeting', value='people who may be similar to their customers')
Row(col_name='target_Region', value='the United States')
Row(col_name='target_Website', value='people who have visited their website or used one of their apps')
Row(col_name='target_Age', value='18 and older')
Row(col_name='target_MinAge', value='18')
Row(col_name='target_Region', value='the United States')
Row(col_name='target_Age', value='18 and older')
Row(col_name='target_MinAge', value='18')
Row(col_name='target_Retargeting', value='people who may be similar to their customers')
Row(col_name='target_Region', value='the United States')
Row(col_name='target_Age', value='18 and older')
Row(col_name='target_MinAge', va

In [29]:
df.count()

4146

In [30]:
df.show(5)

+---------------+------------+
|       col_name|       value|
+---------------+------------+
|     target_Age|30 and older|
|  target_MinAge|          30|
|  target_Region|    Colorado|
|target_Language|English (US)|
|     target_Age|18 and older|
+---------------+------------+
only showing top 5 rows



In [24]:
print(column_index)
print(col_group)
print(row_index)
print(value)
print(new_row)

999
['target_Age', 'target_MinAge', 'target_Region', 'target_Retargeting']
3
people who may be similar to their customers
Row(col_name='target_Retargeting', value='people who may be similar to their customers')


In [9]:
df.count()

4146

In [31]:


# Add a unique row identifier
df = df.withColumn("row_id", monotonically_increasing_id())

# Pivot the dataframe to convert it to wide format
wide_df = df.groupBy("row_id").pivot("col_name").agg(first("value"))

# Show the resulting dataframe
wide_df.show(truncate=False)

+------+------------+-------------+-----------+-------------+---------------+---------------+-----------+-----------+-------------+-------------+-----------------+--------------------------------------------+--------------+------------+---------------------------------------------------------------+
|row_id|target_Age  |target_Agency|target_City|target_Gender|target_Interest|target_Language|target_Like|target_List|target_MaxAge|target_MinAge|target_Region    |target_Retargeting                          |target_Segment|target_State|target_Website                                                 |
+------+------------+-------------+-----------+-------------+---------------+---------------+-----------+-----------+-------------+-------------+-----------------+--------------------------------------------+--------------+------------+---------------------------------------------------------------+
|0     |30 and older|null         |null       |null         |null           |null           |null

In [3]:
wide_df.count()

4146

checking for merging merged data

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from bs4 import BeautifulSoup
import html2text
from pyspark.sql.functions import udf, col, size, regexp_replace, split, expr, concat_ws
from pyspark.sql.types import IntegerType, FloatType
import sys
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import explode, count, row_number, when, avg, round
from pyspark.sql.types import BooleanType, FloatType
from pyspark.sql.window import Window
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


## file_path = "/home/dhwang/Project2/fbpac-ads-en-US.csv"

file_path = "/home/dhwang/Project2/fb_monolith.csv"

spark = SparkSession.builder.master("local[*]").appName("Prep").getOrCreate()

df = spark.read.csv(file_path, header=True, inferSchema=True, multiLine=True, sep=',', escape='"',
                    ignoreLeadingWhiteSpace=True)

def count_words(text):
    if text is not None:
        return len(text.split())
    else:
        return 0

# Define UDF for counting words
count_words_udf = udf(count_words, IntegerType())

# Add new column for word count
df3 = df.withColumn("word_count", count_words_udf("ad_text"))

# Remove non-word characters and split the message into words
df3 = df3.withColumn("words", split(regexp_replace(col("ad_text"), "[^\w\s]", ""), " "))

# Calculate the average word length
df3 = df3.withColumn("avg_word_length", expr("aggregate(words, 0, (acc, word) -> acc + length(word), acc -> acc / size(words))"))


def sentiment_score(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment = analyzer.polarity_scores(text)
    return sentiment["compound"]

sentiment_score_udf = udf(sentiment_score, returnType=FloatType())

df4 = df3.withColumn("sentiment_score", sentiment_score_udf("ad_text"))

df5 = df4.withColumn("words", concat_ws(",", col("words").cast("string")))



23/05/12 15:05:42 WARN Utils: Your hostname, DSGPU05 resolves to a loopback address: 127.0.1.1; using 10.10.11.64 instead (on interface eno1)
23/05/12 15:05:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/12 15:05:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df3.show(5)

+-------------+--------------------+---------------+-----------+--------------------+--------------------+--------------+--------------------+----------+--------------------+------------------+
|        ad_id|           page_name|political_value|paid_for_by|             ad_text|         observed_at|call_to_action|          targetings|word_count|               words|   avg_word_length|
+-------------+--------------------+---------------+-----------+--------------------+--------------------+--------------+--------------------+----------+--------------------+------------------+
|6210317780298| Pew Research Center|            1.0|       null|If they had the c...|2020-12-25 22:46:...|          null|{'AGE_GENDER': [{...|        54|[If, they, had, t...| 5.333333333333333|
|6213767259936|Doctors Without B...|            1.0|       null|South Sudan is ex...|2020-12-25 02:21:...|          null|{'AGE_GENDER': [{...|       177|[South, Sudan, is...| 5.543859649122807|
|6215414936136|Doctors Without

In [6]:
df4 = df3.drop("words")

In [7]:
df4.write.csv("adobserver.csv", header=True)

In [5]:
df4.show(5)

23/05/10 14:59:27 ERROR Executor: Exception in task 0.0 in stage 5.0 (TID 5)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_1391063/3079845277.py", line 45, in sentiment_score
  File "/home/dhwang/.local/lib/python3.8/site-packages/vaderSentiment/vaderSentiment.py", line 241, in polarity_scores
    for chr in text:
TypeError: 'NoneType' object is not iterable

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:556)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:86)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:68)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:509)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_1391063/3079845277.py", line 45, in sentiment_score
  File "/home/dhwang/.local/lib/python3.8/site-packages/vaderSentiment/vaderSentiment.py", line 241, in polarity_scores
    for chr in text:
TypeError: 'NoneType' object is not iterable


In [7]:
file_path = "/home/dhwang/Project2/fb_monolith.csv"

spark = SparkSession.builder.master("local[*]").appName("Prep").getOrCreate()

df = spark.read.csv(file_path, header=True, inferSchema=True, multiLine=True, sep=',', escape='"',
                    ignoreLeadingWhiteSpace=True)


def html_to_text(html):
    soup = BeautifulSoup(html, 'html.parser')
    return soup.get_text()

html_to_text_udf = udf(html_to_text)

df2 = df.withColumn("ad_text", html_to_text_udf(df["ad_text"]))

In [8]:
df.show(5)

+-------------+--------------------+---------------+-----------+--------------------+--------------------+--------------+--------------------+
|        ad_id|           page_name|political_value|paid_for_by|             ad_text|         observed_at|call_to_action|          targetings|
+-------------+--------------------+---------------+-----------+--------------------+--------------------+--------------+--------------------+
|6210317780298| Pew Research Center|            1.0|       null|If they had the c...|2020-12-25 22:46:...|          null|{'AGE_GENDER': [{...|
|6213767259936|Doctors Without B...|            1.0|       null|South Sudan is ex...|2020-12-25 02:21:...|          null|{'AGE_GENDER': [{...|
|6215414936136|Doctors Without B...|            1.0|       null|South Sudan is ex...|2020-12-25 21:26:...|          null|{'AGE_GENDER': [{...|
|6221670718027|     Washington Post|            1.0|       null|A special Washing...|2020-12-25 14:08:...|          null|{'AGE_GENDER': [{...|

In [9]:
df2.show(5)

23/05/10 14:50:13 ERROR Executor: Exception in task 0.0 in stage 5.0 (TID 5)/ 1]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_1382225/445997855.py", line 10, in html_to_text
  File "/usr/lib/python3/dist-packages/bs4/__init__.py", line 287, in __init__
    elif len(markup) <= 256 and (
TypeError: object of type 'NoneType' has no len()

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:556)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:86)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:68)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:509)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterato

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_1382225/445997855.py", line 10, in html_to_text
  File "/usr/lib/python3/dist-packages/bs4/__init__.py", line 287, in __init__
    elif len(markup) <= 256 and (
TypeError: object of type 'NoneType' has no len()


In [7]:
data = pd.read_csv("/home/dhwang/Project2/merged_data.csv")

/tmp/ipykernel_3380768/4250438041.py:1: DtypeWarning: Columns (0,9,12,13,14,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/home/dhwang/Project2/merged_data.csv")


In [8]:
data

,id,advertiser,political_probability,paid_for_by,message,created_at,targeting,updated_at,source,html,...,lang,images,impressions,targets,entities,page,lower_page,targetings,targetedness,listbuilding_fundraising_proba
0,6210317780298,Pew Research Center,1.000000,NaN,"If they had the chance, what would Biden and T...",2020-12-25 22:46:24,"{'AGE_GENDER': [{'Age': '6-53', 'Gender': 'ANY...",2020-12-31 00:00:00,adObserver,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6213767259936,Doctors Without Borders/Médecins Sans Frontièr...,1.000000,NaN,South Sudan is experiencing multiple crises at...,2020-12-25 02:21:58,"{'AGE_GENDER': [{'Age': '6-53', 'Gender': 'ANY...",2021-01-28 00:00:00,adObserver,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6215414936136,Doctors Without Borders/Médecins Sans Frontièr...,1.000000,NaN,South Sudan is experiencing multiple crises at...,2020-12-25 21:26:12,"{'AGE_GENDER': [{'Age': '13-53', 'Gender': 'AN...",2021-01-26 00:00:00,adObserver,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6221670718027,Washington Post,1.000000,NaN,"A special Washington Post report, revealing th...",2020-12-25 14:08:58,"{'AGE_GENDER': [{'Age': '6-53', 'Gender': 'ANY...",2020-12-28 00:00:00,adObserver,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6224247278266,World Wildlife Fund,1.000000,NaN,"This holiday season, don't forget about the wi...",2020-12-25 05:44:17,"{'AGE_GENDER': [{'Age': '6-53', 'Gender': 'ANY...",2020-12-25 00:00:00,adObserver,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284792,23843108782710078,Keep Them Accountable,0.999962,"HOUSE MAJORITY PAC, (202) 849-6052, AND PRIORI...",<p>Rep. Katko voted for tax breaks for his wea...,2018-10-19 14:31:52,"<div><div class=""_4-i0 _26c5""><div class=""clea...",2018-10-22 15:40:06,ProPublica,"<div class=""_5pcr userContentWrapper""><div cla...",...,en-US,{https://pp-facebook-ads.s3.amazonaws.com/v/t1...,1.0,"[{""target"": ""Age"", ""segment"": ""18 and older""},...","[{""entity"": ""Katko"", ""entity_type"": ""Person""}]",https://www.facebook.com/KeepThemAccountable18/,https://www.facebook.com/keepthemaccountable18/,"{""<div><div class=\""_4-i0 _26c5\""><div class=\...",7.0,0.116965
284793,23843034525850259,National Republican Congressional Committee,0.999972,the NRCC and not authorized by any candidate o...,<p>Illinois early voting is open NOW &amp; you...,2018-10-25 00:41:42,"<div><div class=""_4-i0 _26c5""><div class=""clea...",2018-10-25 00:41:42,ProPublica,"<div class=""_5pcr userContentWrapper""><div cla...",...,en-US,{https://pp-facebook-ads.s3.amazonaws.com/v/t1...,1.0,"[{""target"": ""List""}, {""target"": ""Age"", ""segmen...","[{""entity"": ""Illinois"", ""entity_type"": ""Region...",https://www.facebook.com/NRCC/,https://www.facebook.com/nrcc/,"{""<div><div class=\""_4-i0 _26c5\""><div class=\...",4.0,0.312412
284794,23842997138670612,POW Action Fund,0.997301,Protect Our Winters Action Fund,<p>From your favorite peaks to the polling pla...,2018-10-10 00:03:32,"<div><div class=""_4-i0 _26c5""><div class=""clea...",2018-10-10 00:03:32,ProPublica,"<div class=""_5pcr userContentWrapper""><div cla...",...,en-US,{https://pp-facebook-ads.s3.amazonaws.com/v/t4...,1.0,"[{""target"": ""Segment"", ""segment"": ""US politics...",[],https://www.facebook.com/POWActionFund/,https://www.facebook.com/powactionfund/,"{""<div><div class=\""_4-i0 _26c5\""><div class=\...",4.0,0.205220
284795,hyperfeed_story_id_5c8b16b11b8f86515960964,Beto O'Rourke,0.999966,Beto for America,<p>Beto just announced he’s running for presid...,2019-03-15 07:07:40,NaN,2019-03-22 21:01:05,ProPublica,"<div class=""_5pa- userContentWrapper""><div cla...",...,en-US,{https://pp-facebook-ads.s3.amazonaws.com/v/t4...,1.0,[],"[{""entity"": ""Beto"", ""entity_type"": ""Region""}]",https://www.facebook.com/betoorourke/,https://www.facebook.com/betoorourke/,NaN,NaN,0.999994


In [15]:
data.iloc[16,4]

' საქართველო ღვინის სამშობლოდ არის წოდებული, სწორედ ამიტომ ქართული სუფრა ღვინის გარეშე წარმოუდგენელია ჩვენი უნიკალური ღვინის აქსესუარების ნაკრები საშუალებას მოგცემთ, რომ ღვინის სერვირების და დეგუსტაციის პროცესი უფრო სასიამოვნო და კომფორტული გახადოთ შეიძინეთ ნაკრები 300₾-ის ნაცვლად 150₾-ად და მიიღეთ კახური საფერავის ღვინო საჩუქრად ნა… See More\r\n საქართველო ღვინის სამშობლოდ არის წოდებული, სწორედ ამიტომ ქართული სუფრა ღვინის გარეშე წარმოუდგენელია\r\n ჩვენი უნიკალური ღვინის აქსესუარების ნაკრები საშუალებას მოგცემთ, რომ ღვინის სერვირების და დეგუსტაციის პროცესი უფრო სასიამოვნო და კომფორტული გახადოთ\r\n შეიძინეთ ნაკრები 300₾-ის ნაცვლად 150₾-ად და მიიღეთ კახური საფერავის ღვინო საჩუქრად \r\nნა… See More\r\nalavi.ge\r\nღვინის აქსესუარების ნაკრები \r\nღვინის აქსესუარების ნაკრები \r\nღვინის აქსესუარების ნაკრები არის ქეისი რომელშიც მოთავსებულია ღვინის დამხმარე აქსესუარები. აქსესუარების ნაკრე...\r\nShop Now\r\nLike\r\nComment'

In [16]:
html_to_text(data.iloc[16,4])

' საქართველო ღვინის სამშობლოდ არის წოდებული, სწორედ ამიტომ ქართული სუფრა ღვინის გარეშე წარმოუდგენელია ჩვენი უნიკალური ღვინის აქსესუარების ნაკრები საშუალებას მოგცემთ, რომ ღვინის სერვირების და დეგუსტაციის პროცესი უფრო სასიამოვნო და კომფორტული გახადოთ შეიძინეთ ნაკრები 300₾-ის ნაცვლად 150₾-ად და მიიღეთ კახური საფერავის ღვინო საჩუქრად ნა… See More\r\n საქართველო ღვინის სამშობლოდ არის წოდებული, სწორედ ამიტომ ქართული სუფრა ღვინის გარეშე წარმოუდგენელია\r\n ჩვენი უნიკალური ღვინის აქსესუარების ნაკრები საშუალებას მოგცემთ, რომ ღვინის სერვირების და დეგუსტაციის პროცესი უფრო სასიამოვნო და კომფორტული გახადოთ\r\n შეიძინეთ ნაკრები 300₾-ის ნაცვლად 150₾-ად და მიიღეთ კახური საფერავის ღვინო საჩუქრად \r\nნა… See More\r\nalavi.ge\r\nღვინის აქსესუარების ნაკრები \r\nღვინის აქსესუარების ნაკრები \r\nღვინის აქსესუარების ნაკრები არის ქეისი რომელშიც მოთავსებულია ღვინის დამხმარე აქსესუარები. აქსესუარების ნაკრე...\r\nShop Now\r\nLike\r\nComment'

In [3]:
df2.show(5)

23/05/08 18:30:41 ERROR Executor: Exception in task 0.0 in stage 2.0 (TID 2)/ 1]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_3380768/1530788430.py", line 26, in html_to_text
  File "/usr/lib/python3/dist-packages/bs4/__init__.py", line 287, in __init__
    elif len(markup) <= 256 and (
TypeError: object of type 'NoneType' has no len()

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:556)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:86)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:68)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:509)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterat

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_3380768/1530788430.py", line 26, in html_to_text
  File "/usr/lib/python3/dist-packages/bs4/__init__.py", line 287, in __init__
    elif len(markup) <= 256 and (
TypeError: object of type 'NoneType' has no len()


In [ ]:
df5.write.csv("merged_with_sent.csv", header=True)